In [1]:
import pandas as pd
import numpy as np

In [2]:
PATH = 'c:\\Users\\calvin\\Documents\\GitHub\\springboard\\champions_league_luck'

In [3]:
games = pd.read_csv(PATH + '/data/processed/games2.csv', index_col=0, dtype={'Round':'category', 'Day':'category', 'season': 'category'}, parse_dates=['Date'])

In [4]:
games.head()

,Round,Day,Date,Notes,season,Wk,PensHome,PensAway,ScoreHome,ScoreAway,HomeTeam,HomeCountry,AwayCountry,AwayTeam,HomeCoef,AwayCoef,med_coef
993,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,2,2,Sporting CP,pt,es,Real Madrid,30.274,99.799,53.963
994,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,3,1,Lyon,fr,nl,Heerenveen,60.363,24.333,53.963
995,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,3,2,Galatasaray,tr,fr,Monaco,51.925,75.363,53.963
996,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,5,0,Rangers,sco,at,Sturm Graz,32.250,26.250,53.963
997,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,0,1,Sparta Prague,cz,eng,Arsenal,43.562,52.727,53.963


In [5]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2628 entries, 993 to 3582
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Round        2628 non-null   category      
 1   Day          2628 non-null   category      
 2   Date         2628 non-null   datetime64[ns]
 3   Notes        509 non-null    object        
 4   season       2628 non-null   category      
 5   Wk           2628 non-null   int64         
 6   PensHome     2628 non-null   int64         
 7   PensAway     2628 non-null   int64         
 8   ScoreHome    2628 non-null   int64         
 9   ScoreAway    2628 non-null   int64         
 10  HomeTeam     2628 non-null   object        
 11  HomeCountry  2628 non-null   object        
 12  AwayCountry  2628 non-null   object        
 13  AwayTeam     2628 non-null   object        
 14  HomeCoef     2628 non-null   float64       
 15  AwayCoef     2628 non-null   float64       
 16  med_

In [6]:
# lets add the coef difference as an absolute value

games['coefdiff'] = games['HomeCoef'] - games['AwayCoef']

In [7]:
games.head()

,Round,Day,Date,Notes,season,Wk,PensHome,PensAway,ScoreHome,ScoreAway,HomeTeam,HomeCountry,AwayCountry,AwayTeam,HomeCoef,AwayCoef,med_coef,coefdiff
993,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,2,2,Sporting CP,pt,es,Real Madrid,30.274,99.799,53.963,-69.525
994,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,3,1,Lyon,fr,nl,Heerenveen,60.363,24.333,53.963,36.030
995,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,3,2,Galatasaray,tr,fr,Monaco,51.925,75.363,53.963,-23.438
996,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,5,0,Rangers,sco,at,Sturm Graz,32.250,26.250,53.963,6.000
997,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,0,1,Sparta Prague,cz,eng,Arsenal,43.562,52.727,53.963,-9.165


In [8]:
# I want to add total goals as well
games['goalstotal'] = games['ScoreHome'] + games['ScoreAway']
games.head()

,Round,Day,Date,Notes,season,Wk,PensHome,PensAway,ScoreHome,ScoreAway,HomeTeam,HomeCountry,AwayCountry,AwayTeam,HomeCoef,AwayCoef,med_coef,coefdiff,goalstotal
993,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,2,2,Sporting CP,pt,es,Real Madrid,30.274,99.799,53.963,-69.525,4
994,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,3,1,Lyon,fr,nl,Heerenveen,60.363,24.333,53.963,36.030,4
995,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,3,2,Galatasaray,tr,fr,Monaco,51.925,75.363,53.963,-23.438,5
996,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,5,0,Rangers,sco,at,Sturm Graz,32.250,26.250,53.963,6.000,5
997,First group stage,Tue,2000-09-12,NaN,2000-2001,1,0,0,0,1,Sparta Prague,cz,eng,Arsenal,43.562,52.727,53.963,-9.165,1


In [9]:
conds = [
    games.ScoreHome > games.ScoreAway,
    games.ScoreHome < games.ScoreAway,
    games.ScoreHome == games.ScoreAway
]

values = [games.HomeTeam, games.AwayTeam, 'tie']

games['game_winner'] = np.select(conds, values)

In [10]:
# Part of what we are trying to find is away goal luck, which means we need to figure out how to combine home and away matches.
# i think i'll need a col with the teams invovled
def teamorder(x):
    if (x.HomeCoef < 1) or (x.AwayCoef < 1) or (x.HomeCoef == x.AwayCoef):
        if x.HomeTeam < x.AwayTeam:
            return x.HomeTeam + "," + x.AwayTeam
        else:
            return x.AwayTeam + "," + x.HomeTeam
    if x.HomeCoef > x.AwayCoef:
        return x.HomeTeam + "," + x.AwayTeam
    if x.HomeCoef < x.AwayCoef:
        return x.AwayTeam + "," + x.HomeTeam

games['teams'] = games.apply(lambda x: teamorder(x), axis=1)

In [11]:
games[['teamA', 'teamB']] = games['teams'].str.split(',', expand=True)

In [12]:

# First I want to look at how each season is structured because they have changed the format a few times. 
uniqueGroups = games.groupby('season')['Round'].unique()
for idx, v in uniqueGroups.items():
    print(idx, v)

2000-2001 ['First group stage', 'Second group stage', 'Quarter-finals', 'Semi-finals', 'Final']
Categories (7, object): ['Final', 'First group stage', 'Group stage', 'Quarter-finals', 'Round of 16', 'Second group stage', 'Semi-finals']
2001-2002 ['First group stage', 'Second group stage', 'Quarter-finals', 'Semi-finals', 'Final']
Categories (7, object): ['Final', 'First group stage', 'Group stage', 'Quarter-finals', 'Round of 16', 'Second group stage', 'Semi-finals']
2002-2003 ['First group stage', 'Second group stage', 'Quarter-finals', 'Semi-finals', 'Final']
Categories (7, object): ['Final', 'First group stage', 'Group stage', 'Quarter-finals', 'Round of 16', 'Second group stage', 'Semi-finals']
2003-2004 ['Group stage', 'Round of 16', 'Quarter-finals', 'Semi-finals', 'Final']
Categories (7, object): ['Final', 'First group stage', 'Group stage', 'Quarter-finals', 'Round of 16', 'Second group stage', 'Semi-finals']
2004-2005 ['Group stage', 'Round of 16', 'Quarter-finals', 'Semi-fina

In [13]:
# looks like the first few years where slightly different, but they still had groups and knockout rounds
# lets make sure those first few years where acutally similar. I dont have personal experience because I was too young to pay attention or watch.
rounds = {'Quarter-finals', 'Semi-finals', 'Final'}
seasons = {'2000-2001', '2001-2002', '2002-2003'}
games[games.Round.isin(rounds) & games.season.isin(seasons)]

,Round,Day,Date,Notes,season,Wk,PensHome,PensAway,ScoreHome,ScoreAway,...,AwayTeam,HomeCoef,AwayCoef,med_coef,coefdiff,goalstotal,game_winner,teams,teamA,teamB
1137,Quarter-finals,Tue,2001-04-03,Leg 1 of 2,2000-2001,0,0,0,3,2,...,Real Madrid,51.925,99.799,53.963,-47.874,5,Galatasaray,"Real Madrid,Galatasaray",Real Madrid,Galatasaray
1138,Quarter-finals,Tue,2001-04-03,Leg 1 of 2,2000-2001,0,0,0,0,1,...,Bayern Munich,89.727,103.201,53.963,-13.474,1,Bayern Munich,"Bayern Munich,Manchester Utd",Bayern Munich,Manchester Utd
1139,Quarter-finals,Wed,2001-04-04,Leg 1 of 2,2000-2001,0,0,0,2,1,...,Valencia,52.727,69.799,53.963,-17.072,3,Arsenal,"Valencia,Arsenal",Valencia,Arsenal
1140,Quarter-finals,Wed,2001-04-04,Leg 1 of 2,2000-2001,0,0,0,3,0,...,La Coruña,53.963,53.963,53.963,0.000,3,Leeds United,"La Coruña,Leeds United",La Coruña,Leeds United
1141,Quarter-finals,Tue,2001-04-17,Leg 2 of 2; Valencia won,2000-2001,0,0,0,1,0,...,Arsenal,69.799,52.727,53.963,17.072,1,Valencia,"Valencia,Arsenal",Valencia,Arsenal
1142,Quarter-finals,Tue,2001-04-17,Leg 2 of 2; Leeds United won,2000-2001,0,0,0,2,0,...,Leeds United,53.963,53.963,53.963,0.000,2,La Coruña,"La Coruña,Leeds United",La Coruña,Leeds United
1143,Quarter-finals,Wed,2001-04-18,Leg 2 of 2; Real Madrid won,2000-2001,0,0,0,3,0,...,Galatasaray,99.799,51.925,53.963,47.874,3,Real Madrid,"Real Madrid,Galatasaray",Real Madrid,Galatasaray
1144,Quarter-finals,Wed,2001-04-18,Leg 2 of 2; Bayern Munich won,2000-2001,0,0,0,2,1,...,Manchester Utd,103.201,89.727,53.963,13.474,3,Bayern Munich,"Bayern Munich,Manchester Utd",Bayern Munich,Manchester Utd
1145,Semi-finals,Tue,2001-05-01,Leg 1 of 2,2000-2001,0,0,0,0,1,...,Bayern Munich,99.799,103.201,53.963,-3.402,1,Bayern Munich,"Bayern Munich,Real Madrid",Bayern Munich,Real Madrid
1146,Semi-finals,Wed,2001-05-02,Leg 1 of 2,2000-2001,0,0,0,0,0,...,Valencia,53.963,69.799,53.963,-15.836,0,tie,"Valencia,Leeds United",Valencia,Leeds United


In [14]:
# looks like how i expected, knockout rounds are quarterfinals and semifinals. Finals is only one game and will be checked for other luck. 

In [15]:
# Now let's separate the dfs into rounds - groups, knockouts, finals
k_rounds = {'Round of 16', 'Quarter-finals', 'Semi-finals'}
groups = games[games.Round.str.contains('[gG]roup', regex=True)]
finals = games[games.Round == 'Final']
knockout = games[games.Round.isin(k_rounds)]
knockout

,Round,Day,Date,Notes,season,Wk,PensHome,PensAway,ScoreHome,ScoreAway,...,AwayTeam,HomeCoef,AwayCoef,med_coef,coefdiff,goalstotal,game_winner,teams,teamA,teamB
1137,Quarter-finals,Tue,2001-04-03,Leg 1 of 2,2000-2001,0,0,0,3,2,...,Real Madrid,51.925,99.799,53.963,-47.874,5,Galatasaray,"Real Madrid,Galatasaray",Real Madrid,Galatasaray
1138,Quarter-finals,Tue,2001-04-03,Leg 1 of 2,2000-2001,0,0,0,0,1,...,Bayern Munich,89.727,103.201,53.963,-13.474,1,Bayern Munich,"Bayern Munich,Manchester Utd",Bayern Munich,Manchester Utd
1139,Quarter-finals,Wed,2001-04-04,Leg 1 of 2,2000-2001,0,0,0,2,1,...,Valencia,52.727,69.799,53.963,-17.072,3,Arsenal,"Valencia,Arsenal",Valencia,Arsenal
1140,Quarter-finals,Wed,2001-04-04,Leg 1 of 2,2000-2001,0,0,0,3,0,...,La Coruña,53.963,53.963,53.963,0.000,3,Leeds United,"La Coruña,Leeds United",La Coruña,Leeds United
1141,Quarter-finals,Tue,2001-04-17,Leg 2 of 2; Valencia won,2000-2001,0,0,0,1,0,...,Arsenal,69.799,52.727,53.963,17.072,1,Valencia,"Valencia,Arsenal",Valencia,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577,Quarter-finals,Thu,2020-08-13,NaN,2019-2020,0,0,0,2,1,...,Atlético Madrid,76.500,127.000,76.500,-50.500,3,RB Leipzig,"Atlético Madrid,RB Leipzig",Atlético Madrid,RB Leipzig
3578,Quarter-finals,Fri,2020-08-14,NaN,2019-2020,0,0,0,2,8,...,Bayern Munich,138.000,128.000,76.500,10.000,10,Bayern Munich,"Barcelona,Bayern Munich",Barcelona,Bayern Munich
3579,Quarter-finals,Sat,2020-08-15,NaN,2019-2020,0,0,0,1,3,...,Lyon,106.000,61.500,76.500,44.500,4,Lyon,"Manchester City,Lyon",Manchester City,Lyon
3580,Semi-finals,Tue,2020-08-18,NaN,2019-2020,0,0,0,0,3,...,Paris S-G,76.500,103.000,76.500,-26.500,3,Paris S-G,"Paris S-G,RB Leipzig",Paris S-G,RB Leipzig


In [16]:
# now i need to split the kockout rounds into matchups
matchup_grouped = knockout.set_index(['season', 'Round', 'teams']).groupby(['season', 'Round', 'teams'])


In [17]:
knockout[knockout.PensHome > 0]

,Round,Day,Date,Notes,season,Wk,PensHome,PensAway,ScoreHome,ScoreAway,...,AwayTeam,HomeCoef,AwayCoef,med_coef,coefdiff,goalstotal,game_winner,teams,teamA,teamB
1707,Quarter-finals,Wed,2005-04-13,Leg 2 of 2; PSV Eindhoven won on penalty kicks...,2004-2005,0,4,2,1,1,...,Lyon,65.247,70.947,67.5110,-5.700,2,tie,"Lyon,PSV Eindhoven",Lyon,PSV Eindhoven
1961,Semi-finals,Tue,2007-05-01,Leg 2 of 2; Liverpool won on penalty kicks fol...,2006-2007,0,4,1,1,0,...,Chelsea,105.950,79.950,79.9500,26.000,1,Liverpool,"Liverpool,Chelsea",Liverpool,Chelsea
2071,Round of 16,Tue,2008-03-04,Leg 2 of 2; Fenerbahçe won on penalty kicks fo...,2007-2008,0,2,3,3,2,...,Fenerbahçe,87.374,36.791,78.8080,50.583,5,Sevilla,"Sevilla,Fenerbahçe",Sevilla,Fenerbahçe
2073,Round of 16,Wed,2008-03-05,Leg 2 of 2; Schalke 04 won on penalty kicks fo...,2007-2008,0,1,4,1,0,...,Schalke 04,89.107,60.640,78.8080,28.467,1,Porto,"Porto,Schalke 04",Porto,Schalke 04
2199,Round of 16,Wed,2009-03-11,Leg 2 of 2; Arsenal won on penalty kicks follo...,2008-2009,0,6,7,1,0,...,Arsenal,81.934,110.996,81.9340,-29.062,1,Roma,"Arsenal,Roma",Arsenal,Roma
2570,Round of 16,Wed,2012-03-07,Leg 2 of 2; APOEL FC won on penalty kicks foll...,2011-2012,0,4,3,1,0,...,Lyon,13.124,92.735,81.3190,-79.611,1,APOEL FC,"Lyon,APOEL FC",Lyon,APOEL FC
2587,Semi-finals,Wed,2012-04-25,Leg 2 of 2; Bayern Munich won on penalty kicks...,2011-2012,0,1,3,2,1,...,Bayern Munich,103.465,118.887,81.3190,-15.422,3,Real Madrid,"Bayern Munich,Real Madrid",Bayern Munich,Real Madrid
2947,Round of 16,Tue,2015-03-17,Leg 2 of 2; Atlético Madrid won on penalty kic...,2014-2015,0,3,2,1,0,...,Leverkusen,119.542,70.328,80.3435,49.214,1,Atlético Madrid,"Atlético Madrid,Leverkusen",Atlético Madrid,Leverkusen
3073,Round of 16,Tue,2016-03-15,Leg 2 of 2; Atlético Madrid won on penalty kic...,2015-2016,0,8,7,0,0,...,PSV Eindhoven,120.999,58.195,95.1020,62.804,0,tie,"Atlético Madrid,PSV Eindhoven",Atlético Madrid,PSV Eindhoven


In [18]:
matchups_dict = {}
for idx, i in matchup_grouped:
    matchups_dict[idx] = i

test = matchups_dict[('2011-2012', 'Round of 16', 'Lyon,APOEL FC')]
test

Day       Date  \
season    Round       teams                           
2011-2012 Round of 16 Lyon,APOEL FC  Tue 2012-02-14   
                      Lyon,APOEL FC  Wed 2012-03-07   

                                                                                 Notes  \
season    Round       teams                                                              
2011-2012 Round of 16 Lyon,APOEL FC                                         Leg 1 of 2   
                      Lyon,APOEL FC  Leg 2 of 2; APOEL FC won on penalty kicks foll...   

                                     Wk  PensHome  PensAway  ScoreHome  \
season    Round       teams                                              
2011-2012 Round of 16 Lyon,APOEL FC   0         0         0          1   
                      Lyon,APOEL FC   0         4         3          1   

                                     ScoreAway  HomeTeam HomeCountry  \
season    Round       teams                                            
2011-2012 Round of 16 Lyon,APOEL FC          0      Lyon          fr   
                      Lyon,APOEL FC          0  APOEL FC          cy   

                                    AwayCountry  AwayTeam  HomeCoef  AwayCoef  \
season    Round       teams                                                     
2011-2012 Round of 16 Lyon,APOEL FC          cy  APOEL FC    92.735    13.124   
                      Lyon,APOEL FC          fr      Lyon    13.124    92.735   

                                     med_coef  coefdiff  goalstotal  \
season    Round       teams                                           
2011-2012 Round of 16 Lyon,APOEL FC    81.319    79.611           1   
                      Lyon,APOEL FC    81.319   -79.611           1   

                                    game_winner teamA     teamB  
season    Round       teams                                      
2011-2012 Round of 16 Lyon,APOEL FC        Lyon  Lyon  APOEL FC  
                      Lyon,APOEL FC    APOEL FC  Lyon  APOEL FC

In [19]:
len(matchups_dict)

257

In [20]:
matchup_cols = ['season', 'round', 'team_A', 'team_B', 'A_homescore', 'B_homescore', 'A_awayscore', 'B_awayscore', 'A_total', 'B_total',
 'game_1_hometeam', 'game_1_winner', 'game_2_winner', 'game_1_total', 'game_2_total', 'coefdiff', 'A_pens', 'B_pens', 'win_cond', 'winner']
matchups = pd.DataFrame(columns=matchup_cols)

In [21]:
test = test.reset_index().sort_values('Date')

In [22]:
test

,season,Round,teams,Day,Date,Notes,Wk,PensHome,PensAway,ScoreHome,...,AwayCountry,AwayTeam,HomeCoef,AwayCoef,med_coef,coefdiff,goalstotal,game_winner,teamA,teamB
0,2011-2012,Round of 16,"Lyon,APOEL FC",Tue,2012-02-14,Leg 1 of 2,0,0,0,1,...,cy,APOEL FC,92.735,13.124,81.319,79.611,1,Lyon,Lyon,APOEL FC
1,2011-2012,Round of 16,"Lyon,APOEL FC",Wed,2012-03-07,Leg 2 of 2; APOEL FC won on penalty kicks foll...,0,4,3,1,...,fr,Lyon,13.124,92.735,81.319,-79.611,1,APOEL FC,Lyon,APOEL FC


In [23]:
test.season[0], test.Round[0], 

('2011-2012', 'Round of 16')

In [24]:
teamA = test.teamA[0]
teamB = test.teamB[0]

In [25]:
# A_homescore
a_homescore = test[test.HomeTeam == test.teamA]['ScoreHome'].item()

In [26]:
# B_homescore
b_homescore = test[test.HomeTeam == test.teamB]['ScoreHome'].item()

In [27]:
# A_awayscore
a_awayscore = test[test.AwayTeam == test.teamA]['ScoreAway'].item()

In [28]:
# B_awayscore
b_awayscore = test[test.AwayTeam == test.teamB]['ScoreAway'].item()

In [29]:
game_1_homescore = test.ScoreHome[0]
game_2_homescore = test.ScoreHome[0]

game_1_hometeam = test.HomeTeam[0]
game_1_hometeam

'Lyon'

In [30]:
game_1_winner = test.game_winner[0]
game_2_winner = test.game_winner[1]
game_1_winner, game_2_winner

('Lyon', 'APOEL FC')

In [31]:

a_total = a_awayscore + a_homescore
b_total = b_awayscore + b_homescore
a_total, b_total

(1, 1)

In [32]:
game_1_total = test.loc[0].ScoreHome + test.loc[0].ScoreAway
game_2_total = test.loc[1].ScoreHome + test.loc[1].ScoreAway
game_1_total, game_2_total

(1, 1)

In [33]:
coefdiff = round(abs(test.loc[0].coefdiff), 3)
coefdiff

79.611

In [34]:
def pens_and_coef(game2):
    if game2.HomeTeam == game2.teamA:
        return game2.PensHome, game2.PensAway, round(game2.HomeCoef, 3), round(game2.AwayCoef, 3)
    else:
        return game2.PensAway, game2.PensHome, round(game2.AwayCoef, 3), round(game2.HomeCoef, 3)
game2 = test.loc[1]
A_pens, B_pens, A_coef, B_coef = pens_and_coef(game2)
A_pens, B_pens, A_coef, B_coef

(3, 4, 92.735, 13.124)

In [35]:
def determine_winner(teamA, teamB, A_pens, B_pens, a_total, b_total, a_awayscore, b_awayscore):
    if A_pens > 0:
        win_cond = 'pens'
        if A_pens > B_pens:
            winner = teamA
            loser = teamB
        else:
            winner = teamB
            loser = teamA
    elif a_total == b_total:
        win_cond = 'away'
        if a_awayscore > b_awayscore:
            winner = teamA
            loser = teamB
        else:
            winner = teamB
            loser = teamA
    # maybe there are cases that went past away goal and before pens, but i dont think that's true... ill check later
    else:
        win_cond = 'total'
        if a_total > b_total:
            winner = teamA
            loser = teamB
        else:
            winner = teamB
            loser = teamA
    return win_cond, winner, loser
win_cond, winner, loser = determine_winner(teamA, teamB, A_pens, B_pens, a_total, b_total, a_awayscore, b_awayscore)

In [36]:
line = {
    'season': test.season[0],
    'round': test.Round[0],
    'team_A': teamA,
    'team_B': teamB,
    'A_homescore': a_homescore,
    'B_homescore': b_homescore,
    'A_awayscore': a_awayscore,
    'B_awayscore': b_awayscore,
    'A_total': a_total,
    'B_total': b_total,
    'game_1_hometeam': game_1_hometeam,
    'game_1_total': game_1_total,
    'game_2_total': game_2_total,
    'game_1_winner': game_1_winner, 
    'game_2_winner': game_2_winner,
    'A_coef': A_coef,
    'B_coef': B_coef,
    'coefdiff': coefdiff,
    'A_pens': A_pens,
    'B_pens': B_pens,
    'win_cond': win_cond,
    'winner': winner

}
matchups.append(line, ignore_index=True)

,season,round,team_A,team_B,A_homescore,B_homescore,A_awayscore,B_awayscore,A_total,B_total,...,game_2_winner,game_1_total,game_2_total,coefdiff,A_pens,B_pens,win_cond,winner,A_coef,B_coef
0,2011-2012,Round of 16,Lyon,APOEL FC,1,1,0,0,1,1,...,APOEL FC,1,1,79.611,3,4,pens,APOEL FC,92.735,13.124


In [37]:
# OK I've test all of the varaibles i want for the matchup dataframe

# reseting matchups
matchups = pd.DataFrame(columns=matchup_cols)
matchups

,season,round,team_A,team_B,A_homescore,B_homescore,A_awayscore,B_awayscore,A_total,B_total,game_1_hometeam,game_1_winner,game_2_winner,game_1_total,game_2_total,coefdiff,A_pens,B_pens,win_cond,winner


In [38]:
# looping through the groupby objects
def create_matchups(matchups, matchups_dict):
    for k in matchups_dict:
        v = matchups_dict[k]
        v = v.reset_index().sort_values('Date')
        
        teamA = v.teamA[0]
        teamB = v.teamB[0]
        

        a_homescore = v[v.HomeTeam == v.teamA]['ScoreHome'].item()
        b_homescore = v[v.HomeTeam == v.teamB]['ScoreHome'].item()
        a_awayscore = v[v.AwayTeam == v.teamA]['ScoreAway'].item()
        b_awayscore = v[v.AwayTeam == v.teamB]['ScoreAway'].item()

        a_total = a_awayscore + a_homescore
        b_total = b_awayscore + b_homescore

        game_1_hometeam = v.HomeTeam[0]

        game_1_winner = v.game_winner[0]
        game_2_winner = v.game_winner[1]

        game_1_total = v.loc[0].ScoreHome + v.loc[0].ScoreAway
        game_2_total = v.loc[1].ScoreHome + v.loc[1].ScoreAway

        coefdiff = round(
            abs(v.loc[0].coefdiff), 
            3)

        game2 = v.loc[1]
        A_pens, B_pens, A_coef, B_coef = pens_and_coef(game2)

        win_cond, winner, loser = determine_winner(teamA, teamB, A_pens, B_pens, a_total, b_total, a_awayscore, b_awayscore)
        
        line = {
        'season': v.season[0],
        'round': v.Round[0],
        'team_A': teamA,
        'team_B': teamB,
        'A_homescore': a_homescore,
        'B_homescore': b_homescore,
        'A_awayscore': a_awayscore,
        'B_awayscore': b_awayscore,
        'A_total': a_total,
        'B_total': b_total,
        'game_1_hometeam': game_1_hometeam,
        'game_1_total': game_1_total,
        'game_2_total': game_2_total,
        'game_1_winner': game_1_winner, 
        'game_2_winner': game_2_winner, 
        'A_coef': A_coef,
        'B_coef': B_coef,
        'coefdiff': coefdiff,
        'A_pens': A_pens,
        'B_pens': B_pens,
        'win_cond': win_cond,
        'winner': winner,
        'loser': loser

        }
        matchups = matchups.append(line, ignore_index=True)
    return matchups

# Oh yeah, I already forgot. Because of the pandemic there was only on game played in the quarter and semi finals for year 2019-2000
# I suppose the best way is to note them and exclude them from the away goal luck part

In [39]:
for k in matchups_dict:
    if len(matchups_dict[k]) == 1:
        print(k)

('2019-2020', 'Quarter-finals', 'Atlético Madrid,RB Leipzig')
('2019-2020', 'Quarter-finals', 'Barcelona,Bayern Munich')
('2019-2020', 'Quarter-finals', 'Manchester City,Lyon')
('2019-2020', 'Quarter-finals', 'Paris S-G,Atalanta')
('2019-2020', 'Semi-finals', 'Bayern Munich,Lyon')
('2019-2020', 'Semi-finals', 'Paris S-G,RB Leipzig')


In [40]:
# removing 1 game knockouts
for k in list(matchups_dict.keys()):
    if len(matchups_dict[k]) == 1:
        del matchups_dict[k]

In [41]:
matchups = create_matchups(matchups, matchups_dict)

In [42]:
matchups.T

,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,250
season,2000-2001,2000-2001,2000-2001,2000-2001,2000-2001,2000-2001,2001-2002,2001-2002,2001-2002,2001-2002,...,2018-2019,2018-2019,2019-2020,2019-2020,2019-2020,2019-2020,2019-2020,2019-2020,2019-2020,2019-2020
round,Quarter-finals,Quarter-finals,Quarter-finals,Quarter-finals,Semi-finals,Semi-finals,Quarter-finals,Quarter-finals,Quarter-finals,Quarter-finals,...,Semi-finals,Semi-finals,Round of 16,Round of 16,Round of 16,Round of 16,Round of 16,Round of 16,Round of 16,Round of 16
team_A,Bayern Munich,La Coruña,Real Madrid,Valencia,Bayern Munich,Valencia,Barcelona,Liverpool,Manchester Utd,Real Madrid,...,Barcelona,Tottenham,Atalanta,Atlético Madrid,Barcelona,Bayern Munich,Juventus,Paris S-G,Real Madrid,Tottenham
team_B,Manchester Utd,Leeds United,Galatasaray,Arsenal,Real Madrid,Leeds United,Panathinaikos,Leverkusen,La Coruña,Bayern Munich,...,Liverpool,Ajax,Valencia,Liverpool,Napoli,Chelsea,Lyon,Dortmund,Manchester City,RB Leipzig
A_homescore,2,2,3,1,2,3,3,1,3,2,...,3,0,4,1,3,4,2,2,1,0
B_homescore,0,3,3,2,0,0,1,4,0,2,...,4,2,3,2,1,0,1,2,2,3
A_awayscore,1,0,2,1,1,0,0,2,2,1,...,0,3,4,3,1,3,0,1,1,0
B_awayscore,1,0,0,0,1,0,1,0,2,0,...,0,1,1,0,1,1,1,0,2,1
A_total,3,2,5,2,3,3,3,3,5,3,...,3,3,8,4,4,7,2,3,2,0
B_total,1,3,3,2,1,0,2,4,2,2,...,4,3,4,2,2,1,2,2,4,4


In [43]:
# save games
games.to_csv(PATH + '/data/processed/games3.csv')

In [44]:
# save matchups
matchups.to_csv(PATH + '/data/processed/matchups.csv')